In [209]:
try:
    h2o.remove_all()
    h2o.cluster().shutdown()
except:
    pass

H2O session _sid_bbb3 closed.


In [207]:
import nltk
from nltk import word_tokenize
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
from nltk.corpus import stopwords
from nltk import word_tokenize
from h2o.automl import H2OAutoML
import numpy as np
import pandas as pd
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from sklearn.kernel_approximation import RBFSampler

In [111]:
#helper functions
#get stop words
#nltk.download()
STOP_WORDS=list(stopwords.words('english'))

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

    


In [113]:
h2o.init(min_mem_size='29G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Lanier\AppData\Local\Temp\tmpkq3in215
  JVM stdout: C:\Users\Lanier\AppData\Local\Temp\tmpkq3in215\h2o_Lanier_started_from_python.out
  JVM stderr: C:\Users\Lanier\AppData\Local\Temp\tmpkq3in215\h2o_Lanier_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_Lanier_aiekq9
H2O cluster total nodes:,1
H2O cluster free memory:,27.79 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


#### Load data

In [114]:

#load data

train_data=h2o.import_file('train.csv')
#test_data=h2o.import_file('test.csv',header=1) #kagle provided this but we won't use it unless we want to submit


Parse progress: |█████████████████████████████████████████████████████████| 100%


#### Vectorize Text (Sibi already did this, this can be skipped)

In [59]:
#load pretrained embeddings
gl=h2o.import_file('glove.6B.300d.txt')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [63]:

#This converts sentenses into vectors

words_train=tokenize(train_data['question_text'])
words_test=tokenize(train_data['question_text'])
w2v_model = H2OWord2vecEstimator(pre_trained=gl)
w2v_model.train(training_frame=words_train)
train_questions= w2v_model.transform(words_train, aggregate_method = "AVERAGE")
#test_questions= w2v_model.transform(words_test, aggregate_method = "AVERAGE") #kaggle gave this file for its competition we don't need it
train_questions=train_questions.cbind(train_data['target'])
del train_data #free up ram


word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [67]:
h2o.export_file(train_questions,"vectorized_train.csv",force=True)
#h2o.export_file(test_questions,"vectorized_test.csv",force=True) #kaggle gave this file for its competition we don't need it

Export File progress: |███████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%


In [175]:
h2o.remove_all()


#### Train models

In [176]:
train_questions=h2o.import_file("train_glove480.csv") #use pretrained by Sibi instead
#train_questions=h2o.import_file("vectorized_train.csv") #use pretrained by Sibi instead
#test_questions=h2o.import_file("vectorized_test.csv") #kaggle gave this file for its competition we don't need it
train_questions


Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257,C258,C259,C260,C261,C262,C263,C264,C265,C266,C267,C268,C269,C270,C271,C272,C273,C274,C275,C276,C277,C278,C279,C280,C281,C282,C283,C284,C285,C286,C287,C288,C289,C290,C291,C292,C293,C294,C295,C296,C297,C298,C299,C300,C301,C302
00002165364db923c7e6,0.0510106,0.00532269,0.00755154,-0.0295723,0.245782,-0.0820922,-0.0341224,0.0510615,-0.0168528,2.36619,-0.264331,-0.0935255,0.0593031,-0.0726281,0.0512527,-0.012673,-0.00064738,0.670599,-0.108071,-0.152699,0.133436,-0.167754,0.0637519,0.00876994,-0.0673889,0.114964,-0.110361,0.148749,-0.12667,0.083761,0.0287516,0.15943,0.0502208,0.0773172,0.189724,-0.0678288,-0.0469008,0.0153311,-0.193426,-0.00298523,0.00544435,0.178721,0.0755265,-0.117596,0.13214,-0.0909577,-0.0442642,0.0459688,0.0472374,0.0183408,-0.210162,0.205587,-0.0262212,-0.0387465,0.159077,0.0748414,0.0663811,-0.0861276,0.129072,-0.0524235,-0.166051,-0.0227882,-0.0936252,0.188334,0.0332548,0.145394,-0.0623804,0.00390968,0.120383,0.0301137,-0.0244256,0.159174,0.042682,-0.0779583,0.162933,0.00124177,0.00349442,-0.0498627,0.0383422,0.164524,-0.0431332,0.176004,-0.00732792,-0.0421539,0.0640984,-0.144935,-0.312476,-0.26345,0.0135457,0.120775,-0.275555,0.15162,0.0195971,0.0873649,0.19298,-0.006572,0.0765897,0.059612,-0.0100935,-0.0550236,-0.131147,0.121238,0.0829547,-0.0487254,0.15724,-1.03149,0.0669909,-0.0637764,0.0266144,0.0291574,-0.0186974,-0.0372635,0.0119358,-0.0974339,-0.0559585,-0.0308403,0.110768,0.0789191,-0.154074,0.00651798,0.0214982,-0.0360729,-0.15425,0.0840733,0.0442981,0.0911856,0.119896,0.0221807,0.0891914,-0.0266729,-0.0838842,0.0100425,-0.0358176,-0.00042508,0.160823,-0.0226856,-0.051957,0.0159455,0.0367605,-0.069799,-0.994596,0.01978,0.2334,-0.0553575,0.0377172,-0.0619433,0.00513015,0.144915,0.0256216,-0.0176644,-0.00756514,0.2013,0.0227414,-0.0489881,-0.0606911,0.0251843,-0.0752936,-0.0310748,-0.114389,0.0866806,-0.0450352,0.160894,-0.048542,0.0495197,-0.0680022,0.0814201,0.112634,-0.160573,0.189174,-0.0610462,0.0200019,0.0288005,-0.0682022,0.0332253,-0.0331904,-0.0692218,-0.00248346,0.0413618,0.00099592,-0.0944012,0.049238,0.110165,-0.0970975,-0.239509,0.0370639,0.0349803,-0.0248835,-0.0586104,-0.00660677,-0.0899496,-0.0275635,-0.00743977,0.0612449,-0.00754838,0.0139347,0.0280194,0.0726349,-0.0485727,-0.00725315,0.119667,-0.0117509,0.200289,-0.0666595,0.0397169,0.255362,-0.0474539,0.0220977,-0.17343,0.0798993,0.0656131,-0.00518846,0.0470785,-0.0321935,-0.0666891,-0.0790958,0.140139,-0.104769,0.125647,-0.0217403,-0.0548543,0.0605896,0.0143649,0.0714595,-0.0256251,-0.0397749,-0.0650754,0.0721566,0.182636,0.0098881,0.164632,-0.110615,0.158964,0.021538,-0.0701125,-0.0777323,-0.08546,-0.002659,-0.0360412,0.089756,0.0499071,-0.0185205,0.159132,0.151317,-0.0153471,0.168736,-0.131951,0.0147602,-0.112879,-0.208412,-0.184408,0.0867634,0.0692656,0.0286499,0.00183753,0

In [177]:
#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])
train_questions

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257,C258,C259,C260,C261,C262,C263,C264,C265,C266,C267,C268,C269,C270,C271,C272,C273,C274,C275,C276,C277,C278,C279,C280,C281,C282,C283,C284,C285,C286,C287,C288,C289,C290,C291,C292,C293,C294,C295,C296,C297,C298,C299,C300,C301,target
00002165364db923c7e6,0.0510106,0.00532269,0.00755154,-0.0295723,0.245782,-0.0820922,-0.0341224,0.0510615,-0.0168528,2.36619,-0.264331,-0.0935255,0.0593031,-0.0726281,0.0512527,-0.012673,-0.00064738,0.670599,-0.108071,-0.152699,0.133436,-0.167754,0.0637519,0.00876994,-0.0673889,0.114964,-0.110361,0.148749,-0.12667,0.083761,0.0287516,0.15943,0.0502208,0.0773172,0.189724,-0.0678288,-0.0469008,0.0153311,-0.193426,-0.00298523,0.00544435,0.178721,0.0755265,-0.117596,0.13214,-0.0909577,-0.0442642,0.0459688,0.0472374,0.0183408,-0.210162,0.205587,-0.0262212,-0.0387465,0.159077,0.0748414,0.0663811,-0.0861276,0.129072,-0.0524235,-0.166051,-0.0227882,-0.0936252,0.188334,0.0332548,0.145394,-0.0623804,0.00390968,0.120383,0.0301137,-0.0244256,0.159174,0.042682,-0.0779583,0.162933,0.00124177,0.00349442,-0.0498627,0.0383422,0.164524,-0.0431332,0.176004,-0.00732792,-0.0421539,0.0640984,-0.144935,-0.312476,-0.26345,0.0135457,0.120775,-0.275555,0.15162,0.0195971,0.0873649,0.19298,-0.006572,0.0765897,0.059612,-0.0100935,-0.0550236,-0.131147,0.121238,0.0829547,-0.0487254,0.15724,-1.03149,0.0669909,-0.0637764,0.0266144,0.0291574,-0.0186974,-0.0372635,0.0119358,-0.0974339,-0.0559585,-0.0308403,0.110768,0.0789191,-0.154074,0.00651798,0.0214982,-0.0360729,-0.15425,0.0840733,0.0442981,0.0911856,0.119896,0.0221807,0.0891914,-0.0266729,-0.0838842,0.0100425,-0.0358176,-0.00042508,0.160823,-0.0226856,-0.051957,0.0159455,0.0367605,-0.069799,-0.994596,0.01978,0.2334,-0.0553575,0.0377172,-0.0619433,0.00513015,0.144915,0.0256216,-0.0176644,-0.00756514,0.2013,0.0227414,-0.0489881,-0.0606911,0.0251843,-0.0752936,-0.0310748,-0.114389,0.0866806,-0.0450352,0.160894,-0.048542,0.0495197,-0.0680022,0.0814201,0.112634,-0.160573,0.189174,-0.0610462,0.0200019,0.0288005,-0.0682022,0.0332253,-0.0331904,-0.0692218,-0.00248346,0.0413618,0.00099592,-0.0944012,0.049238,0.110165,-0.0970975,-0.239509,0.0370639,0.0349803,-0.0248835,-0.0586104,-0.00660677,-0.0899496,-0.0275635,-0.00743977,0.0612449,-0.00754838,0.0139347,0.0280194,0.0726349,-0.0485727,-0.00725315,0.119667,-0.0117509,0.200289,-0.0666595,0.0397169,0.255362,-0.0474539,0.0220977,-0.17343,0.0798993,0.0656131,-0.00518846,0.0470785,-0.0321935,-0.0666891,-0.0790958,0.140139,-0.104769,0.125647,-0.0217403,-0.0548543,0.0605896,0.0143649,0.0714595,-0.0256251,-0.0397749,-0.0650754,0.0721566,0.182636,0.0098881,0.164632,-0.110615,0.158964,0.021538,-0.0701125,-0.0777323,-0.08546,-0.002659,-0.0360412,0.089756,0.0499071,-0.0185205,0.159132,0.151317,-0.0153471,0.168736,-0.131951,0.0147602,-0.112879,-0.208412,-0.184408,0.0867634,0.0692656,0.0286499,0.00183753

In [178]:
#convert target to category

train_questions['target']=train_questions['target'].ascharacter().asfactor()

In [179]:
len(train_questions)

1306118

In [180]:
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.0001
print(sample_percent_increment*len(train_questions))

129.3074


In [187]:
x=train_questions.columns
y='target'
x.remove(y)





test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator()
DL_model = H2ODeepLearningEstimator()
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    model_glm=glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_gbm=gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_DL=DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment
    
    

glm Model Build progress: |███████████████████████████████████████████████| 100%


C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\job.py:70: UserWarning: Reached maximum number of iterations 50!
  warnings.warn(w)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.58157
dtype: float64
GBM acc:predict    0.941659
dtype: float64
DL acc:predict    0.941736
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |████████████████████████████

glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.892288
dtype: float64
GBM acc:predict    0.943116
dtype: float64
DL acc:predict    0.941046
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.867525
dtype: float64
GBM acc:predict    0.943576
dtype: float64
DL acc:predict    0.934453
dtype: float64
glm Model Build

deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.922493
dtype: float64
GBM acc:predict    0.943652
dtype: float64
DL acc:predict    0.946335
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.912527
dtype: float64
GBM acc:predict    0.940892
dtype: float64
DL acc:predict    0.941966
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Mo

In [191]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)

[predict    0.58157
dtype: float64, predict    0.686063
dtype: float64, predict    0.898804
dtype: float64, predict    0.885695
dtype: float64, predict    0.882858
dtype: float64, predict    0.901871
dtype: float64, predict    0.896964
dtype: float64, predict    0.897884
dtype: float64, predict    0.886691
dtype: float64, predict    0.893131
dtype: float64, predict    0.892364
dtype: float64, predict    0.884085
dtype: float64, predict    0.882398
dtype: float64, predict    0.892288
dtype: float64, predict    0.867525
dtype: float64, predict    0.887151
dtype: float64, predict    0.872968
dtype: float64, predict    0.872125
dtype: float64, predict    0.885618
dtype: float64, predict    0.896964
dtype: float64, predict    0.901794
dtype: float64, predict    0.909767
dtype: float64, predict    0.91061
dtype: float64, predict    0.909614
dtype: float64, predict    0.907774
dtype: float64, predict    0.918277
dtype: float64, predict    0.922493
dtype: float64, predict    0.912527
dtype: fl

In [193]:
train_questions=h2o.import_file("train_paragram.csv") 

#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])

#convert target to category
train_questions['target']=train_questions['target'].ascharacter().asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [196]:
train_questions
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.0001

In [197]:
x=train_questions.columns
y='target'
x.remove(y)





test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator()
DL_model = H2ODeepLearningEstimator()
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    model_glm=glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_gbm=gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_DL=DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment
    

glm Model Build progress: |███████████████████████████████████████████████| 100%


C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\job.py:70: UserWarning: Reached maximum number of iterations 50!
  warnings.warn(w)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.559948
dtype: float64
GBM acc:predict    0.937648
dtype: float64
DL acc:predict    0.936808
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████

glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.897123
dtype: float64
GBM acc:predict    0.938793
dtype: float64
DL acc:predict    0.940395
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.879264
dtype: float64
GBM acc:predict    0.938258
dtype: float64
DL acc:predict    0.939708
dtype: float64
glm Model Build

deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.91666
dtype: float64
GBM acc:predict    0.942532
dtype: float64
DL acc:predict    0.939708
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.914905
dtype: float64
GBM acc:predict    0.942761
dtype: float64
DL acc:predict    0.94383
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Mode

In [198]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)

[predict    0.559948
dtype: float64, predict    0.618332
dtype: float64, predict    0.861177
dtype: float64, predict    0.893307
dtype: float64, predict    0.900404
dtype: float64, predict    0.890788
dtype: float64, predict    0.900252
dtype: float64, predict    0.891857
dtype: float64, predict    0.902083
dtype: float64, predict    0.88598
dtype: float64, predict    0.887736
dtype: float64, predict    0.899336
dtype: float64, predict    0.893307
dtype: float64, predict    0.897123
dtype: float64, predict    0.879264
dtype: float64, predict    0.888804
dtype: float64, predict    0.895596
dtype: float64, predict    0.885217
dtype: float64, predict    0.877814
dtype: float64, predict    0.889873
dtype: float64, predict    0.896894
dtype: float64, predict    0.906663
dtype: float64, predict    0.900481
dtype: float64, predict    0.909105
dtype: float64, predict    0.914829
dtype: float64, predict    0.918645
dtype: float64, predict    0.91666
dtype: float64, predict    0.914905
dtype: fl

In [201]:
train_questions=h2o.import_file("train_google_news.csv") 

#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])

#convert target to category
train_questions['target']=train_questions['target'].ascharacter().asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [202]:
print(train_questions)
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.0001

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257,C258,C259,C260,C261,C262,C263,C264,C265,C266,C267,C268,C269,C270,C271,C272,C273,C274,C275,C276,C277,C278,C279,C280,C281,C282,C283,C284,C285,C286,C287,C288,C289,C290,C291,C292,C293,C294,C295,C296,C297,C298,C299,C300,C301,target
00002165364db923c7e6,0.0358665,0.0532338,0.0699546,0.149974,-0.0193128,-0.0505871,0.0197643,-0.0683705,0.05549,0.0297921,-0.0323098,-0.061959,-0.0293357,0.0226052,-0.160445,0.0502264,0.0235318,0.0896225,0.0333141,-0.0434626,-0.0187673,0.0170898,-0.006786,0.0132328,0.0111722,0.0112305,-0.0646071,0.0387407,0.0525887,0.0216841,0.0221058,0.0405606,-0.0885343,0.0229159,0.0159274,0.0346735,0.00844921,0.0612405,0.117787,0.0211119,0.0548096,-0.0269276,0.180753,-0.00846724,-0.0116466,0.0420269,-0.0432684,0.0436235,-0.0301569,0.0266668,0.0411821,0.00524347,-0.0113026,0.0351784,-0.0631103,0.0523286,-0.16798,0.0291415,0.00010542,0.0230158,-0.00502708,0.0802335,-0.0941037,-0.0977672,-0.0174949,0.00558472,-0.0147206,0.0499822,0.018287,0.0699019,0.0332392,0.122337,0.084284,0.0017395,-0.0151839,-0.145755,0.125577,0.121438,0.0335985,0.120078,-0.0278931,-0.0307506,5.549e-05,-0.00843395,-0.0580555,-0.109437,-0.0940469,0.0748624,-0.0389904,0.0214677,-0.00425027,-0.0406272,-0.0354614,-0.0540217,-0.0424916,0.0143599,0.0324513,-0.0677823,0.00533225,-0.00926902,0.00419044,-0.0219616,-0.064501,0.025349,-0.00576782,-0.0569291,-0.0539752,-0.0155418,0.102012,-0.00957975,-0.0800892,0.0486145,-0.0129783,0.0529591,0.0792347,0.0369818,0.0324291,-0.109602,0.0012207,0.0618397,-0.112529,-0.00822587,-0.142195,-0.0267001,-0.016657,-0.00708008,-0.0979503,-0.104679,-0.100974,-0.0150035,0.0180719,-0.0326316,-0.0769043,0.0171675,-0.0101429,-0.032848,0.034127,-0.0346347,-0.00796786,0.0547208,0.0330256,-0.0324541,0.0264615,0.0622392,0.111311,0.058738,-0.00598145,-0.059254,-0.056503,0.018631,0.0636097,0.0362022,-0.0874578,0.00644753,0.0520408,-0.0310614,-0.01542,-0.0457098,0.0366766,0.00219727,0.0137496,0.0469416,-0.00434737,0.0210627,0.0207187,-0.0583135,0.0328855,-0.0208934,0.0703735,0.00439453,-0.169616,-0.0105979,0.00701349,-0.10448,0.00752198,-0.00074872,0.0885787,-0.0150923,0.0121793,-0.0576852,-0.0813876,-0.0959459,0.0112527,-0.0678877,-0.0116078,0.0140658,-0.0303282,0.0282093,0.103894,0.134288,0.0846948,-0.0286546,0.0228438,-0.00153143,-0.0482733,0.0138439,0.0259788,0.0827554,-0.017112,-0.127785,-0.0192621,0.0482525,0.00693165,-0.0497215,0.0108532,-0.00847834,0.0206521,-0.0842285,-0.00817178,-0.016313,-0.0176773,0.0267112,-0.0105785,0.0395966,-0.16515,-0.0243045,0.0319769,-0.0102539,-0.113129,0.00295743,0.0239369,-0.00663619,-0.0123888,-0.0603249,0.00760165,-0.0604581,0.0612238,0.0749845,0.0498713,-0.0393788,0.0683372,-0.0595398,0.020813,0.0499711,0.00340479,-0.0282482,0.0242809,-0.034876,0.111883,-0.048731,0.0125576,-0.0444447,0.0326815,-0.0434126,0.0638483,-0.0176003,0.00264671,0.020

In [203]:
x=train_questions.columns
y='target'
x.remove(y)


test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator()
DL_model = H2ODeepLearningEstimator()
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    model_glm=glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_gbm=gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    model_DL=DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment

glm Model Build progress: |███████████████████████████████████████████████| 100%


C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\job.py:70: UserWarning: Reached maximum number of iterations 50!
  warnings.warn(w)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.605431
dtype: float64
GBM acc:predict    0.939149
dtype: float64
DL acc:predict    0.93638
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |████████████████████████████

glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.888991
dtype: float64
GBM acc:predict    0.94138
dtype: float64
DL acc:predict    0.942226
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.876991
dtype: float64
GBM acc:predict    0.94138
dtype: float64
DL acc:predict    0.940611
dtype: float64
glm Model Build p

deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.914224
dtype: float64
GBM acc:predict    0.942919
dtype: float64
DL acc:predict    0.938226
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
GLM acc:predict    0.916763
dtype: float64
GBM acc:predict    0.942149
dtype: float64
DL acc:predict    0.941688
dtype: float64
glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Mo

In [204]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)

[predict    0.605431
dtype: float64, predict    0.630279
dtype: float64, predict    0.869759
dtype: float64, predict    0.906454
dtype: float64, predict    0.911609
dtype: float64, predict    0.88376
dtype: float64, predict    0.890453
dtype: float64, predict    0.898992
dtype: float64, predict    0.84545
dtype: float64, predict    0.902377
dtype: float64, predict    0.899069
dtype: float64, predict    0.882683
dtype: float64, predict    0.88653
dtype: float64, predict    0.888991
dtype: float64, predict    0.876991
dtype: float64, predict    0.878606
dtype: float64, predict    0.892838
dtype: float64, predict    0.873836
dtype: float64, predict    0.889453
dtype: float64, predict    0.895146
dtype: float64, predict    0.902069
dtype: float64, predict    0.9043
dtype: float64, predict    0.909993
dtype: float64, predict    0.911609
dtype: float64, predict    0.919301
dtype: float64, predict    0.912147
dtype: float64, predict    0.914224
dtype: float64, predict    0.916763
dtype: float

#### Kernal techniques

In [237]:

#look at using RBF kernal
train_questions=pd.read_csv("train_google_news.csv", header=-1)


In [238]:
try:
    train_questions=train_questions.drop([301], axis=1)
except:
    pass    
try:
    train_questions=train_questions.drop([0], axis=1)
except:
    pass

In [239]:
train_questions

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,0.035866,0.053234,0.069955,0.149974,-0.019313,-0.050587,0.019764,-0.068370,0.055490,0.029792,...,-0.066363,-0.049205,-0.094255,0.013644,-0.062549,0.021157,-0.028675,-0.014383,0.132319,-0.024961
1,0.072091,0.067841,0.023289,0.186210,-0.100337,0.032348,0.048686,-0.004691,-0.005827,0.029109,...,-0.101166,0.075518,-0.055165,0.044503,0.025226,-0.013035,0.060645,0.008981,-0.009111,0.007812
2,0.086572,0.054761,0.057257,0.035168,-0.152283,0.032202,0.073047,-0.074487,0.109106,0.055774,...,-0.150049,0.093408,-0.075439,-0.100317,0.035132,0.119238,0.132129,-0.025014,-0.008398,-0.072852
3,0.161325,0.114990,0.005929,-0.034860,-0.110531,0.021955,0.055943,-0.187919,0.049177,-0.008807,...,-0.072405,-0.011012,-0.083051,0.070164,-0.039934,-0.006278,0.084978,-0.029550,0.038225,-0.014509
4,-0.023340,0.028647,0.024878,0.130652,-0.064862,0.031525,0.007111,-0.075439,0.049875,0.066040,...,-0.041943,0.084326,-0.128595,0.077607,-0.022559,-0.021344,-0.003259,-0.024292,-0.017212,0.004449
5,0.011148,-0.024257,-0.009931,0.143886,-0.123134,0.003871,0.049652,-0.092843,-0.026716,0.064017,...,-0.091378,0.087272,-0.052482,0.000872,-0.049020,0.098300,-0.079575,-0.007638,0.059873,0.053641
6,0.062690,-0.033655,0.094290,0.118157,-0.097509,-0.008448,0.035761,-0.009133,0.138295,0.003274,...,-0.026888,-0.037702,-0.033679,0.021233,-0.012048,0.030321,0.108998,-0.060683,0.015836,0.014246
7,0.017563,-0.025154,0.044691,0.146536,-0.126683,0.089142,-0.013238,-0.033630,0.075845,0.080484,...,-0.059222,0.083133,-0.132486,0.002267,-0.054868,0.010088,-0.041374,-0.043265,0.082905,0.018045
8,0.039295,0.019814,0.010188,0.093365,-0.058144,0.058197,0.025261,-0.051811,0.088110,0.042071,...,-0.101952,0.013939,-0.076649,0.078011,-0.037309,0.074863,0.075032,0.004190,0.090431,0.037584
9,0.060757,0.026429,0.024071,0.128674,-0.110606,0.007019,0.043392,-0.054585,0.027582,0.061210,...,-0.087682,0.065775,-0.108148,0.004391,-0.036356,-0.013926,0.010197,-0.055198,0.038910,-0.020619


In [242]:
train_questions=train_questions.astype('float32')
train_questions=train_questions.dropna()

In [243]:
rbf_feature=RBFSampler(gamma=1, random_state=1)
X_features=rbf_feature.fit_transform(train_questions)

In [246]:
X_features=pd.DataFrame(X_features)
X_features

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.088141,0.130277,-0.139081,0.130217,0.027880,-0.057184,0.141419,0.139879,-0.134426,0.127033,...,0.016256,0.002739,-0.128933,-0.103790,0.069402,-0.141344,-0.134283,0.014804,0.131012,0.063134
1,-0.096247,-0.101821,-0.133772,-0.012895,-0.097496,-0.038785,0.122597,0.056843,-0.117823,0.055597,...,0.118550,-0.097176,0.056082,-0.115359,-0.109400,0.091542,-0.074367,-0.107919,0.062475,-0.076342
2,0.078144,-0.112481,-0.109683,0.136714,-0.135746,-0.108182,0.132896,-0.068066,-0.118740,-0.106830,...,0.034732,-0.141420,-0.113836,0.129370,-0.126277,-0.137885,0.089546,0.066291,0.139016,0.121694
3,-0.055473,-0.055991,-0.050735,-0.141234,-0.131245,-0.063670,0.140593,-0.081799,-0.018982,0.137551,...,-0.093682,-0.118346,-0.111936,0.141124,0.008090,-0.062423,0.094366,0.092015,-0.038329,0.021327
4,-0.131732,-0.131182,-0.129942,-0.085537,-0.077793,-0.140533,0.133416,-0.083746,-0.113101,0.067510,...,-0.088774,0.017556,-0.020047,0.123413,0.021208,-0.102262,-0.094288,0.136891,0.132819,-0.125035
5,0.010593,-0.084433,0.077219,-0.019940,-0.141058,0.127154,0.140227,0.116188,-0.140936,0.141321,...,-0.141082,-0.067351,0.074516,-0.090518,-0.103858,-0.138813,-0.141144,0.141397,0.132675,0.066982
6,0.034884,0.141108,-0.136544,-0.036760,0.019850,-0.057504,0.132565,0.012746,-0.135451,-0.052377,...,0.133579,0.082644,0.141001,-0.139797,-0.126921,-0.013562,-0.131224,0.054460,0.139434,0.125373
7,-0.009700,-0.134521,-0.112464,0.061949,-0.123796,-0.138338,-0.048084,-0.116412,0.085897,0.032401,...,-0.002296,0.053482,0.137990,0.038222,0.060696,0.031827,-0.098102,-0.138229,0.072816,0.079207
8,-0.124028,-0.032750,0.042289,-0.103782,-0.107562,-0.141181,0.140449,0.052127,-0.139872,0.141101,...,0.056948,-0.104374,0.141421,-0.072694,-0.139664,0.109204,0.074246,-0.077316,0.065909,0.127132
9,-0.132866,-0.110982,-0.129151,0.041848,-0.118118,-0.031259,0.025091,-0.103805,0.122394,0.140997,...,-0.091961,-0.123986,0.060519,-0.051184,-0.109694,-0.067201,-0.094595,0.014476,0.109021,-0.064706


In [252]:
try:
    del train_questions
    del rbf_feature
except:
    pass
Y= pd.read_csv("train_google_news.csv", header=-1,usecols = [301])

In [254]:

train_questions=pd.concat([X_features,Y],axis=1)

In [255]:
train_questions

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,301
0,-0.088141,0.130277,-0.139081,0.130217,0.027880,-0.057184,0.141419,0.139879,-0.134426,0.127033,...,0.002739,-0.128933,-0.103790,0.069402,-0.141344,-0.134283,0.014804,0.131012,0.063134,0
1,-0.096247,-0.101821,-0.133772,-0.012895,-0.097496,-0.038785,0.122597,0.056843,-0.117823,0.055597,...,-0.097176,0.056082,-0.115359,-0.109400,0.091542,-0.074367,-0.107919,0.062475,-0.076342,0
2,0.078144,-0.112481,-0.109683,0.136714,-0.135746,-0.108182,0.132896,-0.068066,-0.118740,-0.106830,...,-0.141420,-0.113836,0.129370,-0.126277,-0.137885,0.089546,0.066291,0.139016,0.121694,0
3,-0.055473,-0.055991,-0.050735,-0.141234,-0.131245,-0.063670,0.140593,-0.081799,-0.018982,0.137551,...,-0.118346,-0.111936,0.141124,0.008090,-0.062423,0.094366,0.092015,-0.038329,0.021327,0
4,-0.131732,-0.131182,-0.129942,-0.085537,-0.077793,-0.140533,0.133416,-0.083746,-0.113101,0.067510,...,0.017556,-0.020047,0.123413,0.021208,-0.102262,-0.094288,0.136891,0.132819,-0.125035,0
5,0.010593,-0.084433,0.077219,-0.019940,-0.141058,0.127154,0.140227,0.116188,-0.140936,0.141321,...,-0.067351,0.074516,-0.090518,-0.103858,-0.138813,-0.141144,0.141397,0.132675,0.066982,0
6,0.034884,0.141108,-0.136544,-0.036760,0.019850,-0.057504,0.132565,0.012746,-0.135451,-0.052377,...,0.082644,0.141001,-0.139797,-0.126921,-0.013562,-0.131224,0.054460,0.139434,0.125373,0
7,-0.009700,-0.134521,-0.112464,0.061949,-0.123796,-0.138338,-0.048084,-0.116412,0.085897,0.032401,...,0.053482,0.137990,0.038222,0.060696,0.031827,-0.098102,-0.138229,0.072816,0.079207,0
8,-0.124028,-0.032750,0.042289,-0.103782,-0.107562,-0.141181,0.140449,0.052127,-0.139872,0.141101,...,-0.104374,0.141421,-0.072694,-0.139664,0.109204,0.074246,-0.077316,0.065909,0.127132,0
9,-0.132866,-0.110982,-0.129151,0.041848,-0.118118,-0.031259,0.025091,-0.103805,0.122394,0.140997,...,-0.123986,0.060519,-0.051184,-0.109694,-0.067201,-0.094595,0.014476,0.109021,-0.064706,0
